<a href="https://colab.research.google.com/github/MROWAtk/PUM/blob/main/Lab7_systemy_rekomendacyjne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [3]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

In [4]:
liczba_ocen = len(ratings)

In [5]:
print(f"Liczba wszystkich ocen: {liczba_ocen}")

Liczba wszystkich ocen: 100836


In [6]:
liczba_filmow = len(movies)

In [7]:
print(f"Liczba filmów w bazie: {liczba_filmow}")

Liczba filmów w bazie: 9742


In [8]:
liczba_uzytkownikow = ratings['userId'].nunique()

In [9]:
print(f"Liczba uzytkownikow: {liczba_uzytkownikow}")

Liczba uzytkownikow: 610


In [10]:
avg_liczba_ocen_na_uzytkownika = round(ratings.groupby('userId').size().mean(), 2)

In [11]:
print(f"Średnia liczba ocen na użytkownika: {avg_liczba_ocen_na_uzytkownika}")

Średnia liczba ocen na użytkownika: 165.3


In [12]:
avg_liczba_ocen_na_film = round(ratings.groupby('movieId').size().mean(), 2)

In [13]:
print(f"Średnia liczba ocen na film: {avg_liczba_ocen_na_film}")

Średnia liczba ocen na film: 10.37


In [14]:
rozkład_ocen = ratings['rating'].value_counts().sort_index()

In [15]:
print(rozkład_ocen)

rating
0.5     1370
1.0     2811
1.5     1791
2.0     7551
2.5     5550
3.0    20047
3.5    13136
4.0    26818
4.5     8551
5.0    13211
Name: count, dtype: int64


In [16]:
movie_titles = dict(zip(movies['movieId'], movies['title']))
C = ratings['rating'].count().mean()
m = ratings['rating'].mean()

In [17]:
def bayesian_avg(ratings):
  bayesian_avg = (C * m + ratings.sum()) / (C + ratings.count())
  return bayesian_avg

In [20]:
bayesian_avg_ratings = ratings.groupby('movieId')['rating'].agg(bayesian_avg).reset_index()
bayesian_avg_ratings.columns = ['movieId', 'bayesian_avg']
movies_with_bayesian = movies.merge(bayesian_avg_ratings, on='movieId')

In [21]:
print("\n5 wierszy danych o filmach z wartością średniej oceny bayesowskiej:")
print(movies_with_bayesian.head())


5 wierszy danych o filmach z wartością średniej oceny bayesowskiej:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  bayesian_avg  
0  Adventure|Animation|Children|Comedy|Fantasy      3.502449  
1                   Adventure|Children|Fantasy      3.501481  
2                               Comedy|Romance      3.501432  
3                         Comedy|Drama|Romance      3.501478  
4                                       Comedy      3.501348  


In [22]:
!pip install numpy==1.23.5

In [23]:
!pip install surprise
from surprise.model_selection import cross_validate, GridSearchCV
from surprise.prediction_algorithms import SVD, KNNBasic, KNNBaseline
from surprise import Dataset
from surprise import Reader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2461562 sha256=bd79425500c38c59b2bebd128235f29e66d5ed35d2018f86781c76519d1b733e
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [24]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()

algo = SVD()
algo.fit(trainset)

all_movie_ids = ratings['movieId'].unique()

rated_movie_ids_user_600 = ratings[ratings['userId'] == 600]['movieId'].tolist()
unrated_movie_ids_user_600 = [movie_id for movie_id in all_movie_ids if movie_id not in rated_movie_ids_user_600]

predictions = []
for movie_id in unrated_movie_ids_user_600:
  predictions.append((movie_id, algo.predict(600, movie_id).est))

predictions.sort(key=lambda x: x[1], reverse=True)

top_10_recommended_movie_ids = [movie_id for movie_id, ratinng in predictions[:10]]

recommended_movies = movies[movies['movieId'].isin(top_10_recommended_movie_ids)]

print("Recommendations for user 600:")
print(recommended_movies[['title']])

Recommendations for user 600:
                                                  title
585   Wallace & Gromit: The Best of Aardman Animatio...
596          Ghost in the Shell (Kôkaku kidôtai) (1995)
685                                      Vertigo (1958)
905                                 12 Angry Men (1957)
906                           Lawrence of Arabia (1962)
2247                               On Any Sunday (1971)
3562                                Donnie Darko (2001)
6016                         Kiss Kiss Bang Bang (2005)
6529                                    Stardust (2007)
7456                                  Black Swan (2010)
